In [8]:
!pip install lxml

In [9]:
!pip install bs4

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1279 sha256=fa905c305e6ea5c41ff31824de9ecf8bf8db698e78c0b5a75c137cb71ca3572f
  Stored in directory: c:\users\8b1d~1\appdata\local\pip\cache\wheels\0a\9e\ba\20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [14]:
!pip install requests

In [1]:
import pandas as pd
from IPython.display import clear_output
import re
import requests as req
from bs4 import BeautifulSoup

## https://news.sarbc.ru/lenta/

Функция, которая ищет ссылку на документ и краткое содержание

In [1]:
def get_data(url):
    dat = []
    link = []
    r = req.get(url)
    soup=BeautifulSoup(r.content,"lxml")
    for i, data in enumerate(soup.find_all('div', 'lenta-news-description')):
        dat.append(re.sub('[\n]','',data.text).strip(' '))
        print(i/(2*len(soup.find_all('div', 'lenta-news-description'))))
        clear_output(True)
    for i,data in enumerate(soup.find_all(rel = 'nofollow', style="line-height: 150%;")):
        link.append(str(data)[8:str(data).find('rel')])
        print(0.5 + i/(2*len(soup.find_all(rel = 'nofollow', style="line-height: 150%;"))))
        clear_output(True)
    data = pd.DataFrame({'link':link, 'data':dat})
    return data.drop(data[data['data'] == ''].index)

In [2]:
sarbc_data = get_data('https://news.sarbc.ru/lenta/')

,link,data
0,"""http://www.rbc.ru/society/15/04/2020/5e96da1a...","В ведомстве заявили, что расследование в отнош..."
1,"""http://www.rbc.ru/society/15/04/2020/5e2fe945...",В России число зараженных коронавирусом превыс...
2,"""http://www.rbc.ru/society/15/04/2020/5e96d457...",Скопление людей в очередях перед входом в метр...
3,"""http://www.rbc.ru/economics/15/04/2020/5e96c3...",Глобальный спрос на нефть в апреле 2020 года у...
4,"""http://www.rbc.ru/business/15/04/2020/5e96c16...","Российская нефть Urals подешевела, несмотря на..."
...,...,...
208,"""https://lenta.ru/news/2020/04/14/rezin/""",В Москве владельцев двухкомнатной квартиры пос...
209,"""https://lenta.ru/news/2020/04/14/gpn/""",Совместное предприятие «Газпром нефти» и Shell...
210,"""https://lenta.ru/news/2020/04/14/russia_usa/""","Российский пловец, трехкратный призер чемпиона..."
211,"""https://lenta.ru/news/2020/04/14/sila/""",В Краснодарском крае врачи силой укладывали кр...


## https://svpressa.ru/all/news-archive/

Функция, которая ищет документ и краткое содержание

In [180]:
from tqdm import tqdm_notebook
import warnings


def get_data_1(pages):
    #Будем использовать библиотеку bs4 для парсинга
    url = 'https://svpressa.ru/all/news-archive/'
    r = req.get(url)
    soup = BeautifulSoup(r.content.decode("utf-8"))
    data_short = []
    data_full = []
    #Каждый раз будем переходить на новую страницу новостей
    for j in tqdm_notebook(range(pages)):
        #Чтобы очищать RAM
        if j%100 == 0:
            df = pd.DataFrame({'data_full':data_full, 'data_short':data_short})
            df.to_csv("/Users/александр/DLS deep learning school/Huawei_project/data/data"+str(j/100)+".csv", index = False)
            data_short = []
            data_full = []
        #Если новостей больше нет-останавливаемся
        if soup.find_all('div', "b-article__container_item") == []:
            break
        #Находим специальные контейнеры в которых находятся новости
        try:
            for i in soup.find_all('div', "b-article__container_item"):
                #Находим и преобразовываем краткое содержание
                data_short.append(re.sub('\xa0',' ',i.find('a', "b-article__title b-article__title_item").text))
                #Ссылка на полную новость
                link = str(i.find('a', "b-article__title b-article__title_item"))
                link = 'https://svpressa.ru/{}/{}/{}'.format(*re.findall(r'\w+', str(link))[7:10])
                #Переходим по ссылке и преобразуем новость
                z = BeautifulSoup(req.get(link).content)\
                .find('div', 'b-text__block b-text__block_text b-text__block_offset_large') 
                raw_text = [re.sub('\xa0', ' ', piece.text) for piece in z.find_all('p')[:-2]]
                data_full.append(raw_text) #попробовать поменять кодировку, чтобы не делать преобразований
            #Обновляем страницу
            page = str(soup.find_all('a', 'b-pages-nav__btn')[0])
            page = page[page.find('href')+6:-12]
            soup = BeautifulSoup(req.get(url + page).content.decode("utf-8"))
        except:
            pass
        

warnings.filterwarnings("ignore")

In [181]:
get_data_1(1000)